<a href="https://colab.research.google.com/github/KelvinY18/Data-Science-WebScraping/blob/main/Project1BeautifulSoup_WebscrapingJobStreet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# WEB SCRAPPING for JOBSTREET

import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

# generating url for postion and location
def get_url(position,location):
    position = position.replace(' ','-')
    location = location.replace(' ','-')
    template = 'https://www.jobstreet.com.my/en/job-search/{}-jobs-in-{}/'
    url = template.format(position,location)
    return url

def get_job(card):
    # job name
    try:
        job_name = card.find('div','sx2jih0 _2j8fZ_0 sIMFL_0 _1JtWu_0').text.strip()
    except:
        job_name = ''

    # job url
    try:
        anchor_tag = card.a
        job_url = 'https://www.jobstreet.com.my'+anchor_tag.get('href')
    except:
        job_url = ''

    # company name
    try:
        company_name = card.find('span','sx2jih0 zcydq82b _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8').text.strip()
    except:
        company_name = ''

    # job location
    try:
        job_location = card.find('span','sx2jih0 zcydq82b zcydq8r iwjz4h0').text.strip()
    except:
        job_location = ''

    # fetching job summary
    try:
     summary = ''
     tag_str = ''
     job_summary = card.find('ul','sx2jih0 sx2jih3 h6p8rp0 h6p8rp5')
     for tag in job_summary("li"):
        tag_str = tag.text
        summary += tag_str + ','
        summary = summary.rstrip(',')
    except AttributeError:
     summary = ''

    # fetching job salary
    job_salary = card.find_all('span', 'sx2jih0 zcydq82b _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc6')

    try:
     if len(job_salary) >= 2:
        job_salary = job_salary[1]
        salary = job_salary.text.strip()
     else:
        salary = ''
    except IndexError:
     salary = ''

    # fetch date
    time_tag = card.time
    post_date = time_tag.get('datetime')
    post_date = post_date.split("T", 1)
    post_date = post_date[0]
    today = datetime.today().strftime('%Y-%m-%d')

    job_info = (job_name,job_url,company_name,job_location,summary,salary,post_date,today)


    return job_info

def main(position,location):
    records = []
    url = get_url(position,location)

    response = requests.get(url)

    soup = BeautifulSoup(response.text,'html.parser')

    cards = soup.find_all('div','sx2jih0 zcydq85k zcydq84t zcydq83t zcydq842')


    for everyCard in cards:
        jobDetails = get_job(everyCard)
        records.append(jobDetails)



    # here we are using panda Data frame to save jobs in a csv file

    col = ['JobName','JobUrl','CompanyName','JobLocation','JobSummary','JobSalary','PostDate','Today']

    JobStreet_data = pd.DataFrame(records,columns=col)

    JobStreet_data.to_csv('D:\\JobStreetData.csv',encoding='utf-8')


In [ ]:
main('Python developer','Selangor')